In [100]:
import sys
sys.path.append("../repository/")
import database as db
import polars as pl

In [101]:
conn_wgi = db.DatabaseConnection("world-governance-index").connection
conn_ti = db.DatabaseConnection("transformation-index").connection
conn_ef = db.DatabaseConnection("economic-freedom").connection
conn_db = db.DatabaseConnection("doing-bussiness").connection




In [102]:
def get_countries() -> pl.DataFrame:
    wgi = pl.read_database(
        query="""
        SELECT "name", "sub-region"
        FROM "iso_codes"
        """,
        connection=conn_wgi,
    )
    return wgi

countries = get_countries()

In [103]:
countries = countries.group_by("sub-region").agg(pl.col("name")).drop_nulls().sort(pl.col("sub-region"))
countries

sub-region,name
str,list[str]
"""Australia and …","[""Australia"", ""Christmas Island"", … ""Norfolk Island""]"
"""Central Asia""","[""Kazakhstan"", ""Kyrgyzstan"", … ""Uzbekistan""]"
"""Eastern Asia""","[""China"", ""Hong Kong"", … ""Taiwan, Province of China""]"
"""Eastern Europe…","[""Belarus"", ""Bulgaria"", … ""Ukraine""]"
"""Latin America …","[""Anguilla"", ""Antigua and Barbuda"", … ""Virgin Islands (U.S.)""]"
"""Melanesia""","[""Fiji"", ""New Caledonia"", … ""Vanuatu""]"
"""Micronesia""","[""Guam"", ""Kiribati"", … ""United States Minor Outlying Islands""]"
"""Northern Afric…","[""Algeria"", ""Egypt"", … ""Western Sahara""]"
"""Northern Ameri…","[""Bermuda"", ""Canada"", … ""United States of America""]"


RED RED RED  Australia and New Zealand
BLUE BLUE BLUE  Australia
BLUE BLUE BLUE  Christmas Island
BLUE BLUE BLUE  Cocos (Keeling) Islands
BLUE BLUE BLUE  Heard Island and McDonald Islands
BLUE BLUE BLUE  New Zealand
BLUE BLUE BLUE  Norfolk Island



RED RED RED  Central Asia
BLUE BLUE BLUE  Kazakhstan
BLUE BLUE BLUE  Kyrgyzstan
BLUE BLUE BLUE  Tajikistan
BLUE BLUE BLUE  Turkmenistan
BLUE BLUE BLUE  Uzbekistan



RED RED RED  Eastern Asia
BLUE BLUE BLUE  China
BLUE BLUE BLUE  Hong Kong
BLUE BLUE BLUE  Japan
BLUE BLUE BLUE  Korea (Democratic People's Republic of)
BLUE BLUE BLUE  Korea, Republic of
BLUE BLUE BLUE  Macao
BLUE BLUE BLUE  Mongolia
BLUE BLUE BLUE  Taiwan, Province of China



RED RED RED  Eastern Europe
BLUE BLUE BLUE  Belarus
BLUE BLUE BLUE  Bulgaria
BLUE BLUE BLUE  Czechia
BLUE BLUE BLUE  Hungary
BLUE BLUE BLUE  Moldova, Republic of
BLUE BLUE BLUE  Poland
BLUE BLUE BLUE  Romania
BLUE BLUE BLUE  Russian Federation
BLUE BLUE BLUE  Slovakia
BLUE BLUE BLUE  Ukraine



RED RED R